<a href="https://colab.research.google.com/github/VictorSilvaCamargo/Atividade_Pratica_2_RA3/blob/main/garcom_cozinheiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import threading
import time

class Garcom(threading.Thread):
    def __init__(self, nome, cozinheiro, lista_pedidos):
        threading.Thread.__init__(self)
        self.nome = nome
        self.cozinheiro = cozinheiro
        self.lista_pedidos = lista_pedidos

    def fazer_pedido(self):
        with self.cozinheiro.condicao:
            while len(self.lista_pedidos) >= 4:
                print(f'{self.nome}: Lista de pedidos cheia. Aguardando vaga...')
                self.cozinheiro.condicao.wait()

            pedido = f'Pedido do {self.nome}'
            self.lista_pedidos.append(pedido)
            print(f'{self.nome}: Fez um pedido: {pedido}. Total de pedidos: {len(self.lista_pedidos)}')

            if len(self.lista_pedidos) == 1:
                self.cozinheiro.condicao.notify_all()

    def run(self):
        for _ in range(10):
            time.sleep(0.5)  # Simula o tempo levado para tomar um pedido
            self.fazer_pedido()

class Cozinheiro(threading.Thread):
    def __init__(self, nome, lista_pedidos):
        threading.Thread.__init__(self)
        self.nome = nome
        self.lista_pedidos = lista_pedidos
        self.condicao = threading.Condition()

    def preparar_pedido(self):
        with self.condicao:
            while len(self.lista_pedidos) == 0:
                self.condicao.wait()

            pedido = self.lista_pedidos.pop(0)
            print(f'{self.nome}: Preparou o pedido: {pedido}. Pedidos restantes: {len(self.lista_pedidos)}')

            if len(self.lista_pedidos) == 3:
                self.condicao.notify_all()

    def run(self):
        while True:
            time.sleep(1)  # Simula o tempo levado para preparar um pedido
            self.preparar_pedido()
            if len(self.lista_pedidos) == 0:
                break

# Criação das instâncias e inicialização das threads
lista_pedidos = []
cozinheiro = Cozinheiro('Cozinheiro', lista_pedidos)
garcons = [Garcom(f'Garçom {i}', cozinheiro, lista_pedidos) for i in range(1, 6)]

# Medição do tempo inicial
tempo_inicial = time.time()

cozinheiro.start()
for garcom in garcons:
    garcom.start()

# Aguarda todas as threads finalizarem
cozinheiro.join()
for garcom in garcons:
    garcom.join()

# Medição do tempo final
tempo_final = time.time()

print('Todos os pedidos foram entregues.')
print(f'Tempo total: {tempo_final - tempo_inicial} segundos.')


Garçom 1: Fez um pedido: Pedido do Garçom 1. Total de pedidos: 1
Garçom 2: Fez um pedido: Pedido do Garçom 2. Total de pedidos: 2
Garçom 3: Fez um pedido: Pedido do Garçom 3. Total de pedidos: 3
Garçom 4: Fez um pedido: Pedido do Garçom 4. Total de pedidos: 4
Garçom 5: Lista de pedidos cheia. Aguardando vaga...
Cozinheiro: Preparou o pedido: Pedido do Garçom 1. Pedidos restantes: 3
Garçom 5: Fez um pedido: Pedido do Garçom 5. Total de pedidos: 4
Garçom 1: Lista de pedidos cheia. Aguardando vaga...
Garçom 2: Lista de pedidos cheia. Aguardando vaga...
Garçom 3: Lista de pedidos cheia. Aguardando vaga...
Garçom 4: Lista de pedidos cheia. Aguardando vaga...
Garçom 5: Lista de pedidos cheia. Aguardando vaga...
Cozinheiro: Preparou o pedido: Pedido do Garçom 2. Pedidos restantes: 3
Garçom 2: Fez um pedido: Pedido do Garçom 2. Total de pedidos: 4
Garçom 5: Lista de pedidos cheia. Aguardando vaga...
Garçom 1: Lista de pedidos cheia. Aguardando vaga...
Garçom 3: Lista de pedidos cheia. Aguardan

In [85]:
import threading
import time

class Garcom(threading.Thread):
    def __init__(self, nome, cozinheiros, lista_pedidos, contador):
        threading.Thread.__init__(self)
        self.nome = nome
        self.cozinheiros = cozinheiros
        self.lista_pedidos = lista_pedidos
        self.contador = contador

    def fazer_pedido(self):
        with self.cozinheiros[0].condicao:
            while len(self.lista_pedidos) >= 4:
                print(f'{self.nome}: Lista de pedidos cheia. Aguardando vaga...')
                self.cozinheiros[0].condicao.wait()

            pedido = f'Pedido do {self.nome}'
            self.lista_pedidos.append(pedido)
            print(f'{self.nome}: Fez um pedido: {pedido}. Total de pedidos: {len(self.lista_pedidos)}')

            if len(self.lista_pedidos) == 1:
                self.cozinheiros[0].condicao.notify_all()
            else:
                self.cozinheiros[0].condicao.notify()

    def run(self):
        for _ in range(10):
            time.sleep(0.5)  # Simula o tempo levado para tomar um pedido
            self.fazer_pedido()

class Cozinheiro(threading.Thread):
    def __init__(self, nome, lista_pedidos, contador):
        threading.Thread.__init__(self)
        self.nome = nome
        self.lista_pedidos = lista_pedidos
        self.condicao = threading.Condition()
        self.contador = contador

    def preparar_pedido(self):
        with self.condicao:
            while len(self.lista_pedidos) == 0:
                self.condicao.wait()

            pedido = self.lista_pedidos.pop(0)
            print(f'{self.nome}: Preparou o pedido: {pedido}. Pedidos restantes: {len(self.lista_pedidos)}')
            self.contador.increment()  # Incrementa o valor do contador

            self.condicao.notify()

    def run(self):
        while True:
            if self.contador.get_value() >= 49:
                break
            time.sleep(1)  # Simula o tempo levado para preparar um pedido
            self.preparar_pedido()

class Contador:
    def __init__(self):
        self.lock = threading.Lock()
        self.value = 0

    def increment(self):
        with self.lock:
            self.value += 1

    def get_value(self):
        with self.lock:
            return self.value

# Criação das instâncias e inicialização das threads
lista_pedidos = []
contador = Contador()
cozinheiros = [Cozinheiro(f'Cozinheiro {i}', lista_pedidos, contador) for i in range(1, 3)]
garcons = [Garcom(f'Garçom {i}', cozinheiros, lista_pedidos, contador) for i in range(1, 6)]

# Medição do tempo inicial
tempo_inicial = time.time()

for cozinheiro in cozinheiros:
    cozinheiro.start()

for garcom in garcons:
    garcom.start()

# Aguarda todas as threads finalizarem
for cozinheiro in cozinheiros:
    cozinheiro.join()

for garcom in garcons:
    garcom.join()

# Medição do tempo final
tempo_final = time.time()

print('Todos os pedidos foram entregues.')
print(f'Tempo total: {tempo_final - tempo_inicial} segundos.')


Garçom 1: Fez um pedido: Pedido do Garçom 1. Total de pedidos: 1
Garçom 4: Fez um pedido: Pedido do Garçom 4. Total de pedidos: 2
Garçom 2: Fez um pedido: Pedido do Garçom 2. Total de pedidos: 3
Garçom 3: Fez um pedido: Pedido do Garçom 3. Total de pedidos: 4
Garçom 5: Lista de pedidos cheia. Aguardando vaga...
Cozinheiro 1: Preparou o pedido: Pedido do Garçom 1. Pedidos restantes: 3
Garçom 5: Fez um pedido: Pedido do Garçom 5. Total de pedidos: 4
Cozinheiro 2: Preparou o pedido: Pedido do Garçom 4. Pedidos restantes: 3
Garçom 2: Fez um pedido: Pedido do Garçom 2. Total de pedidos: 4
Garçom 1: Lista de pedidos cheia. Aguardando vaga...
Garçom 3: Lista de pedidos cheia. Aguardando vaga...
Garçom 4: Lista de pedidos cheia. Aguardando vaga...
Garçom 5: Lista de pedidos cheia. Aguardando vaga...
Garçom 2: Lista de pedidos cheia. Aguardando vaga...
Cozinheiro 1: Preparou o pedido: Pedido do Garçom 2. Pedidos restantes: 3
Garçom 1: Fez um pedido: Pedido do Garçom 1. Total de pedidos: 4
Garço